In [ ]:
# Import libraries
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import glob
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os
import sys


In [ ]:
TESS = "/content/drive/My Drive/kaggle/input/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"
RAV = "/content/drive/My Drive/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
SAVEE = "/content/drive/My Drive/kaggle/input/surrey-audiovisual-expressed-emotion-savee/ALL/"
EMODB = "/content/drive/My Drive/kaggle/input/emodb/wav/"


In [ ]:
# Get the data location for EMODB
dir_list = os.listdir(EMODB)

# parse the filename to get the emotions
emotion=[]
path = []
for i in dir_list:
    if i[-6:-5]=='F':
        emotion.append('Freude')
    elif i[-6:-5]=='W':
        emotion.append('Arger')
    elif i[-6:-5]=='L':
        emotion.append('Langeweile')
    elif i[-6:-5]=='E':
        emotion.append('Ekel')
    elif i[-6:-5]=='N':
        emotion.append('neutral')
    elif i[-6:-5]=='A':
        emotion.append('Angst')
    elif i[-6:-5]=='T':
        emotion.append('Trauer')
    else:
        emotion.append('error')
    path.append(EMODB + i)

# Now check out the label count distribution
emodb_df = pd.DataFrame(emotion, columns = ['labels'])
emodb_df['source'] = 'EMODB'
emodb_df = pd.concat([emodb_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
emodb_df.labels.value_counts()
emodb_df.head()

,labels,source,path
0,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
1,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
2,neutral,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
3,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
4,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...


In [ ]:

emodb_df.labels.value_counts()

Arger         127
Langeweile     81
neutral        79
Freude         71
Angst          69
Trauer         62
Ekel           46
Name: labels, dtype: int64

In [ ]:
#df = pd.concat([SAVEE_df, RAV_df, TESS_df], axis = 0)
#df = pd.concat([RAV_df, SAVEE_df], axis = 0)

emodb_df.to_csv("/content/drive/My Drive/kaggle/input/Data_path_trial9.csv" ,index=False)


In [ ]:

sub_df = emodb_df.loc[emodb_df['labels'] != 'Ekel']

print(sub_df.labels.value_counts())
sub_df.to_csv("/content/drive/My Drive/kaggle/input/Data_path_trial19.csv" ,index=False)

Arger         127
Langeweile     81
neutral        79
Freude         71
Angst          69
Trauer         62
Name: labels, dtype: int64


In [ ]:
# Import our libraries
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import pandas as pd
import os
import IPython.display as ipd  # To play sound in the notebook
# Importing required libraries
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from keras import layers

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob
import os
import pickle
import IPython.display as ipd  # To play sound in the notebook

In [ ]:
# lets pick up the meta-data that we got from our first part of the Kernel
ref = pd.read_csv("/content/drive/My Drive/kaggle/input/Data_path_trial19.csv")
ref.head()

,labels,source,path
0,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
1,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
2,neutral,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
3,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...
4,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...


In [ ]:
# Note this takes a couple of minutes (~10 mins) as we're iterating over 4 datasets
df = pd.DataFrame(columns=['feature'])
df1 = pd.DataFrame(columns=['feature'])
# loop feature extraction over the entire dataset
counter=counter1=0
for index,path in enumerate(ref.path):
    X, sample_rate = librosa.load(path
                                  , res_type='kaiser_fast'
                                  ,duration=2.5
                                  ,sr=44100
                                  ,offset=0.5
                                 )
    sample_rate = np.array(sample_rate)


    mfccs_first = librosa.feature.mfcc(y=X,
                                 sr=sample_rate,
                                 n_mfcc=13)
    mfccs = np.mean(mfccs_first, axis=0)
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=X), axis=0)
    df.loc[counter] = [mfccs]
    df1.loc[counter1] = [zcr]
    counter=counter+1
    counter1=counter1+1
# Check a few records to make sure its processed successfully
print(len(df))
df.head()

489


,feature
0,"[-1.7612203, -5.110916, -17.499926, -12.728250..."
1,"[-0.15411714, -4.184052, -20.010956, -20.82891..."
2,"[-1.2132392, -2.3784957, -5.90626, -7.551967, ..."
3,"[-6.808378, -9.853182, -18.269833, -19.185545,..."
4,"[-20.45626, -17.734709, -19.502829, -21.832008..."


In [ ]:
# Now extract the mean bands to its own feature columns
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df[:5]

,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-1.761220,-5.110916,-17.499926,-12.728251,-11.324615,-12.359241,-13.935461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-0.154117,-4.184052,-20.010956,-20.828911,-21.120638,-21.530209,-20.789478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,neutral,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-1.213239,-2.378496,-5.906260,-7.551967,-9.145247,-14.359626,-19.010939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-6.808378,-9.853182,-18.269833,-19.185545,-19.167366,-18.221760,-17.373699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-20.456261,-17.734709,-19.502829,-21.832008,-23.387789,-21.753059,-15.963073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = pd.concat([df,pd.DataFrame(df1['feature'].values.tolist())],axis=1)
df[:5]

,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-1.761220,-5.110916,-17.499926,-12.728251,-11.324615,-12.359241,-13.935461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-0.154117,-4.184052,-20.010956,-20.828911,-21.120638,-21.530209,-20.789478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,neutral,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-1.213239,-2.378496,-5.906260,-7.551967,-9.145247,-14.359626,-19.010939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-6.808378,-9.853182,-18.269833,-19.185545,-19.167366,-18.221760,-17.373699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-20.456261,-17.734709,-19.502829,-21.832008,-23.387789,-21.753059,-15.963073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# replace NA with 0
df=df.fillna(0)
print(df.shape)
df[:5]

(489, 435)


,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-1.761220,-5.110916,-17.499926,-12.728251,-11.324615,-12.359241,-13.935461,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Arger,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-0.154117,-4.184052,-20.010956,-20.828911,-21.120638,-21.530209,-20.789478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,neutral,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-1.213239,-2.378496,-5.906260,-7.551967,-9.145247,-14.359626,-19.010939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-6.808378,-9.853182,-18.269833,-19.185545,-19.167366,-18.221760,-17.373699,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Freude,EMODB,/content/drive/My Drive/kaggle/input/emodb/wav...,-20.456261,-17.734709,-19.502829,-21.832008,-23.387789,-21.753059,-15.963073,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Split between train and test
X_train, X_test, y_train, y_test = train_test_split(df.drop(['path','labels','source'],axis=1)  #df changed to df2
                                                    , df.labels                                 #df changed to df2
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=42
                                                   )

# Lets see how the data present itself before normalisation
X_train[150:160]

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
414,-2.985143,-4.024633,-4.248375,-4.077415,-6.907206,-6.696192,-6.536884,-7.909963,-10.620881,-9.860979,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
125,4.796303,3.121945,1.595803,0.349394,1.281833,0.847566,-0.084427,-1.181525,-4.677215,-10.009874,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
297,0.795220,-0.175989,-3.293846,-2.126087,-1.466115,-4.174130,-3.263169,-4.015409,-12.525550,-17.037558,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
223,-22.755962,-23.295689,-25.301609,-22.835192,-21.811819,-9.427584,-5.729278,-4.857125,-5.063878,-4.337646,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
411,-13.001016,-15.956806,-14.551130,-14.660938,-15.168959,-14.986892,-16.081135,-19.855854,-27.444061,-29.590769,...,0.033203,0.039062,0.042969,0.045410,0.053223,0.093262,0.143555,0.197266,0.201172,0.149414
219,7.612350,5.001493,-2.387615,-2.029789,-3.284718,-3.805171,-3.718347,-4.411745,-2.841043,-3.773657,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
129,1.849791,1.042299,-2.388429,-2.826522,-2.423001,-2.715855,-2.280817,-1.940734,-2.290709,-2.348060,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
464,-20.644938,-22.368237,-27.095163,-27.830502,-27.149410,-25.749325,-25.133635,-23.365345,-22.046770,-17.809963,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
284,5.520494,4.162608,0.106769,0.727566,0.443556,1.555771,0.181928,-4.967151,-4.442563,-2.155455,...,0.044922,0.051758,0.051758,0.031738,0.021484,0.016602,0.011719,0.009277,0.006836,0.003906
323,1.877573,-2.121683,-10.723438,-11.356007,-10.398555,-9.069937,-10.579392,-8.482162,-7.698379,-5.477535,...,0.004883,0.006348,0.004883,0.003906,0.009277,0.008301,0.008301,0.009766,0.003418,0.000000


In [ ]:
# Lets do data normalization
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# Check the dataset now
X_train[150:160]

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
414,0.316478,0.379452,0.761093,0.785803,0.539807,0.573735,0.586865,0.443891,0.164834,0.227455,...,-0.457883,-0.457672,-0.454947,-0.453646,-0.468640,-0.468694,-0.439868,-0.408892,-0.390313,-0.379489
125,1.037175,1.103675,1.325823,1.218216,1.347639,1.323012,1.230561,1.110547,0.756711,0.212769,...,-0.457883,-0.457672,-0.454947,-0.453646,-0.468640,-0.468694,-0.439868,-0.408892,-0.390313,-0.379489
297,0.666605,0.769468,0.853330,0.976410,1.076560,0.824236,0.913450,0.829765,-0.024835,-0.480412,...,-0.457883,-0.457672,-0.454947,-0.453646,-0.468640,-0.468694,-0.439868,-0.408892,-0.390313,-0.379489
223,-1.514644,-1.573449,-1.273307,-1.046467,-0.930502,0.302441,0.667432,0.746367,0.718207,0.772253,...,-0.457883,-0.457672,-0.454947,-0.453646,-0.468640,-0.468694,-0.439868,-0.408892,-0.390313,-0.379489
411,-0.611166,-0.829737,-0.234475,-0.248002,-0.275198,-0.249732,-0.365269,-0.739712,-1.510436,-1.718607,...,0.548990,0.756373,0.905712,1.036098,1.309383,2.637242,3.784281,4.743695,5.527840,5.603239
219,1.297990,1.294146,0.940900,0.985816,0.897158,0.860883,0.868042,0.790496,0.939559,0.827883,...,-0.457883,-0.457672,-0.454947,-0.453646,-0.468640,-0.468694,-0.439868,-0.408892,-0.390313,-0.379489
129,0.764277,0.892927,0.940821,0.907991,0.982165,0.969078,1.011450,1.035324,0.994362,0.968498,...,-0.457883,-0.457672,-0.454947,-0.453646,-0.468640,-0.468694,-0.439868,-0.408892,-0.390313,-0.379489
464,-1.319126,-1.479462,-1.446620,-1.534412,-1.457044,-1.318701,-1.268345,-1.087433,-0.972968,-0.556599,...,-0.457883,-0.457672,-0.454947,-0.453646,-0.468640,-0.468694,-0.439868,-0.408892,-0.390313,-0.379489
284,1.104248,1.209135,1.181935,1.255156,1.264945,1.393354,1.257133,0.735466,0.780078,0.987495,...,0.904357,1.150938,1.184029,0.587573,0.249094,0.084195,-0.095040,-0.166568,-0.189210,-0.223078
323,0.766850,0.572294,0.135399,0.074826,0.195393,0.337964,0.183585,0.387197,0.455860,0.659820,...,-0.309813,-0.260389,-0.300327,-0.325496,-0.158709,-0.192249,-0.195615,-0.153814,-0.289762,-0.379489


In [ ]:
# Lets few preparation steps to get it into the correct format for Keras

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)
#print(y_train[0:10])
#print(y_test[0:10])

# Pickel the lb object for future use
filename = 'label'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()

(366, 432)
['Angst' 'Arger' 'Freude' 'Langeweile' 'Trauer' 'neutral']


In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape

(366, 432, 1)

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
earlystopping = EarlyStopping(monitor ="val_acc",
                              mode = 'auto', patience = 7,
                              restore_best_weights = True)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=7,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [ ]:
from keras import backend as K
from keras import regularizers

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from keras import models
from tensorflow.keras.layers import LSTM, Bidirectional

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint = ModelCheckpoint("/content/drive/My Drive/kaggle/saved_models/best_model1.hdf5", monitor='val_accuracy', verbose=1,
    save_best_only=True, mode='auto')

In [ ]:


#model =Sequential()
#model.add(layers.Conv1D(512, kernel_size=3,
#                        padding="same", activation="relu",
#                        input_shape=(X_train.shape[1], 1),
 #                       kernel_regularizer=regularizers.l2(l2=0.005),
  #                      bias_regularizer=regularizers.l2(l2=0.005)))
#model.add(layers.BatchNormalization())
#model.add(layers.MaxPool1D(pool_size=5, padding="same"))

#model.add(layers.Conv1D(512, kernel_size=3,
 #                       padding="same", activation="relu",
  #                      kernel_regularizer=regularizers.l2(l2=0.005),
   #                     bias_regularizer=regularizers.l2(l2=0.005)))
#model.add(layers.BatchNormalization())
#model.add(layers.MaxPool1D(pool_size=5,padding="same"))

#model.add(layers.Conv1D(256, kernel_size=3,
 #                       padding="same", activation="relu",
  #                      kernel_regularizer=regularizers.l2(l2=0.005),
   #                     bias_regularizer=regularizers.l2(l2=0.005)))
#model.add(layers.BatchNormalization())
#model.add(layers.MaxPool1D(pool_size=5,  padding="same"))


#model.add(layers.Conv1D(256, kernel_size=3,  padding='same', activation='relu',
 #                       kernel_regularizer=regularizers.l2(l2=0.005),
  #                      bias_regularizer=regularizers.l2(l2=0.005)))
#model.add(layers.BatchNormalization())
#model.add(layers.MaxPooling1D(pool_size=5,  padding = 'same'))

#model.add(layers.Conv1D(128, kernel_size=3,  padding='same', activation='relu',
 #                       kernel_regularizer=regularizers.l2(l2=0.005),
  #                      bias_regularizer=regularizers.l2(l2=0.005)))
#model.add(layers.BatchNormalization())
#model.add(layers.MaxPooling1D(pool_size=3,  padding = 'same'))
#model.add(Bidirectional(LSTM(units=64)))

#model.add(layers.Flatten())
#model.add(layers.Dense(512, activation='relu',
 #                       kernel_regularizer=regularizers.l2(l2=0.005),
  #                      bias_regularizer=regularizers.l2(l2=0.005)))
#model.add(layers.BatchNormalization())
#model.add(layers.Dense(6, activation="softmax"))

#model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc", f1_m])


#New Model
# model = Sequential()

# model.add(Conv1D(512, 5,padding='same',
#                  input_shape=(X_train.shape[1], 1)))
# model.add(Activation('elu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPool1D(pool_size=5, padding="same"))


# model.add(Conv1D(512, 5,padding='same'))
# model.add(Activation('elu'))
# model.add(layers.BatchNormalization())
# model.add(Dropout(0.1))
# model.add(layers.MaxPool1D(pool_size=5,padding="same"))


# model.add(Conv1D(256, 5,padding='same'))
# model.add(Activation('relu'))
# model.add(layers.BatchNormalization())
# model.add(Dropout(0.1))
# model.add(layers.MaxPool1D(pool_size=5,padding="same"))
# #model.add(Conv1D(128, 5,padding='same',))
# #model.add(Activation('relu'))
# #model.add(Conv1D(128, 5,padding='same',))
# #model.add(Activation('relu'))
# #model.add(Dropout(0.2))

# model.add(Conv1D(256, 5,padding='same'))
# model.add(Activation('relu'))
# model.add(layers.BatchNormalization())
# model.add(Dropout(0.1))
# model.add(layers.MaxPool1D(pool_size=5,padding="same"))

# model.add(Conv1D(128, 5,padding='same'))
# model.add(Activation('relu'))
# model.add(layers.BatchNormalization())
# model.add(Dropout(0.1))
# model.add(layers.MaxPool1D(pool_size=5,padding="same"))


# model.add(Conv1D(128, 5,padding='same',))
# model.add(Activation('relu'))
# model.add(Bidirectional(LSTM(units=64)))

# model.add(Flatten())
# model.add(Dense(10))
# model.add(layers.Dense(14, activation="softmax"))

In [ ]:
model =Sequential()
model.add(Conv1D(128, 5, padding='same',
                         input_shape=(X_train.shape[1], 1),
                         kernel_regularizer=regularizers.l2(l2=0.1),
                       bias_regularizer=regularizers.l2(l2=0.1)))
model.add(Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=5, padding="same"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5, padding='same',
                         input_shape=(X_train.shape[1], 1),
                         kernel_regularizer=regularizers.l2(l2=0.1),
                       bias_regularizer=regularizers.l2(l2=0.1)))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, padding='same',
                         input_shape=(X_train.shape[1], 1),
                         kernel_regularizer=regularizers.l2(l2=0.1),
                       bias_regularizer=regularizers.l2(l2=0.1)))
model.add(Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=5, padding="same"))
model.add(Bidirectional(LSTM(units=64)))
model.add(Flatten())
model.add(Dense(6, kernel_regularizer=regularizers.l2(l2=0.1),
                       bias_regularizer=regularizers.l2(l2=0.1)))
model.add(Activation('softmax'))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc", f1_m])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 432, 128)          768       
                                                                 
 activation_3 (Activation)   (None, 432, 128)          0         
                                                                 
 batch_normalization_2 (Batc  (None, 432, 128)         512       
 hNormalization)                                                 
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 87, 128)          0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 87, 128)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 87, 128)          

In [ ]:
EPOCHS = 150
batch_size = 32

In [ ]:
 history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                     epochs=EPOCHS, batch_size=batch_size,
#                      callbacks=[earlystopping, learning_rate_reduction]
                      callbacks=[checkpoint]
                    )
# history = fit_evaluate(EPOCHS = 50,batch_size = 128)

Epoch 1/150
6/6 [==============================] - 8s 522ms/step - loss: 22.6768 - acc: 0.3251 - f1_m: 0.1575 - val_loss: 20.2909 - val_acc: 0.4309 - val_f1_m: 0.0000e+00
Epoch 2/150
6/6 [==============================] - 2s 270ms/step - loss: 18.3652 - acc: 0.4754 - f1_m: 0.3107 - val_loss: 16.4738 - val_acc: 0.4797 - val_f1_m: 0.0000e+00
Epoch 3/150
6/6 [==============================] - 2s 260ms/step - loss: 14.6613 - acc: 0.5109 - f1_m: 0.3257 - val_loss: 13.2332 - val_acc: 0.4715 - val_f1_m: 0.0000e+00
Epoch 4/150
6/6 [==============================] - 2s 270ms/step - loss: 11.5894 - acc: 0.5301 - f1_m: 0.3642 - val_loss: 10.5927 - val_acc: 0.4553 - val_f1_m: 0.0000e+00
Epoch 5/150
6/6 [==============================] - 2s 265ms/step - loss: 9.1120 - acc: 0.5628 - f1_m: 0.3802 - val_loss: 8.5213 - val_acc: 0.3008 - val_f1_m: 0.0000e+00
Epoch 6/150
6/6 [==============================] - 2s 265ms/step - loss: 7.1848 - acc: 0.5902 - f1_m: 0.3983 - val_loss: 6.9297 - val_acc: 0.1870 -